# Airburst Solver

In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from deepimpact import *


In [2]:
# Initialise the Planet class
earth = Planet()


In [3]:
# Solve the atmospheric entry problem for a given set of input parameters
result = earth.solve_atmospheric_entry(
    radius=35, angle=45, strength=1e7, density=3000, velocity=19e3
)
result


,velocity,mass,angle,altitude,distance,radius,time
0,19000.000000,5.387831e+08,45.000000,100000.000000,0.000000,35.000000,0.00
1,19000.346520,5.387831e+08,44.995097,99328.271169,661.439148,35.000000,0.05
2,19000.692984,5.387831e+08,44.990194,98656.587581,1323.015631,35.000000,0.10
3,19001.039390,5.387830e+08,44.985289,97984.949257,1984.729482,35.000000,0.15
4,19001.385735,5.387830e+08,44.980383,97313.356220,2646.580735,35.000000,0.20
...,...,...,...,...,...,...,...
193,4064.805952,9.552442e+07,44.410823,456.728486,100046.371302,259.024295,9.65
194,4059.151270,9.550115e+07,44.414376,314.594919,100191.440721,259.024295,9.70
195,4053.405792,9.547757e+07,44.417936,172.651804,100336.300997,259.024295,9.75
196,4047.568588,9.545367e+07,44.421503,30.902341,100480.948863,259.024295,9.80


In [4]:
# Calculate the kinetic energy lost per unit altitude and add it as a column to the result dataframe
result = earth.calculate_energy(result)
result


,velocity,mass,angle,altitude,distance,radius,time,Ek,dedz
0,19000.000000,5.387831e+08,45.000000,100000.000000,0.000000,35.000000,0.00,9.725036e+16,0.000000
1,19000.346520,5.387831e+08,44.995097,99328.271169,661.439148,35.000000,0.05,9.725390e+16,-1.260046
2,19000.692984,5.387831e+08,44.990194,98656.587581,1323.015631,35.000000,0.10,9.725744e+16,-1.259765
3,19001.039390,5.387830e+08,44.985289,97984.949257,1984.729482,35.000000,0.15,9.726098e+16,-1.259458
4,19001.385735,5.387830e+08,44.980383,97313.356220,2646.580735,35.000000,0.20,9.726452e+16,-1.259125
...,...,...,...,...,...,...,...,...,...
193,4064.805952,9.552442e+07,44.410823,456.728486,100046.371302,259.024295,9.65,7.891582e+14,3.981198
194,4059.151270,9.550115e+07,44.414376,314.594919,100191.440721,259.024295,9.70,7.867723e+14,4.042859
195,4053.405792,9.547757e+07,44.417936,172.651804,100336.300997,259.024295,9.75,7.843529e+14,4.105085
196,4047.568588,9.545367e+07,44.421503,30.902341,100480.948863,259.024295,9.80,7.818997e+14,4.167871


In [5]:
# Determine the outcomes of the impact event
outcome = earth.analyse_outcome(result)
outcome


{'outcome': 'Airburst',
 'burst_peak_dedz': 2903.8291634854527,
 'burst_altitude': 11143.03136561095,
 'burst_distance': 89111.91321957881,
 'burst_energy': 16016.541793214074}

# Damage Mapper

In [17]:
# Calculate the blast location and damage radius for several pressure levels
meteoroid_lat = 55.2
meteoroid_lon = -2.5
pressures = [1e3, 4e3, 30e3, 50e3]

blast_lat, blast_lon, damage_rad = damage_zones(
    outcome, lat=meteoroid_lat, lon=meteoroid_lon, bearing=217.0, pressures=pressures
)


In [18]:
# Create the DataFrame
mapping = MappingFunctions()


In [20]:
# Plot the meteoroid trajectory and damage areas
end_coords = (blast_lat, blast_lon)
start_coords = (meteoroid_lat, meteoroid_lon)
mapping.plot_damage_areas(start_coords, end_coords, damage_rad)


In [21]:
mapping.get_fmap()


In [22]:
# The GeospatialLocator tool
locator = GeospatialLocator()

# Find the postcodes in the damage radii
postcodes = locator.get_postcodes_by_radius(
    (blast_lat, blast_lon), radii=damage_rad
)


In [23]:
# Find the population in each postcode
population = locator.get_population_by_radius(
    (blast_lat, blast_lon), radii=damage_rad
)


In [24]:
# Print the number of people affected in each damage zone
print()
print("Pressure |      Damage | Population")
print("   (kPa) | radius (km) |   affected")
print("-----------------------------------")
for pop, rad, zone in zip(population, damage_rad, pressures):
    print(f"{zone/1e3:8.0f} | {rad/1e3:11.1f} | {pop:10,.0f}")
print()



Pressure |      Damage | Population
   (kPa) | radius (km) |   affected
-----------------------------------
       1 |       154.8 | 13,965,739
       4 |        51.3 |    471,084
      30 |        10.8 |      7,746
      50 |         4.6 |        167



In [25]:
# Print the postcodes inside the highest damage zone
print("Postcodes in the highest damage zone:")
print(*postcodes[-1])
print()


Postcodes in the highest damage zone:
CA130RR CA130RS CA130RT CA130RU CA130SU CA139FG CA139UU CA139UX CA139UY CA139UZ CA139XA



In [26]:
# Example usage of impact_risk function
# Uses the default file impact_parameter_list.csv in the resources folder
probability, population = impact_risk(earth, pressure=30e3)


In [28]:
probability


,postcode,probability
0,CV130AA,0.4
1,CV130AB,0.4
2,CV130AD,0.4
3,CV130AE,0.7
4,CV130AG,0.6
...,...,...
13628,LE9 6TP,0.2
13629,LE9 6TQ,0.2
13630,LE9 6TS,0.1
13631,LE9 6TX,0.2


In [30]:
print(
    "Total population affected: "
    + f"{population['mean']:,.0f} +/- {population['stdev']:,.0f}"
)


Total population affected: 214,392 +/- 131,203
